In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_4630/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_4630/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_3_16_5,0.980665,0.767143,0.318924,0.972701,0.188203,1.557115,2.359925,0.184759,0.996833,0.433823,0.966089,0.437645,101.340473,235.842147,"Hidden Size=[12], regularizer=0.02, learning_r..."
1,model_2_29_9,0.971455,0.759413,0.575293,0.990894,0.277848,1.608805,1.471606,0.074951,0.867378,0.527113,0.959825,0.531757,68.561365,159.144125,"Hidden Size=[8], regularizer=0.02, learning_ra..."
2,model_2_29_0,0.968664,0.756635,0.548838,0.993677,0.305013,1.627385,1.563274,0.052044,1.011576,0.552280,0.955897,0.557146,68.374802,158.957562,"Hidden Size=[8], regularizer=0.02, learning_ra..."
3,model_2_29_2,0.969023,0.756289,0.552146,0.993194,0.301520,1.629695,1.551810,0.056025,1.015974,0.549108,0.956402,0.553946,68.397840,158.980600,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_30_2,0.971240,0.752831,0.556435,0.940299,0.279936,1.652822,1.536949,0.240009,1.092255,0.529090,0.959523,0.533752,68.546385,159.129146,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
474,model_3_40_2,0.977653,-57.608732,0.107265,0.940910,0.217515,391.916863,3.093319,0.475944,0.750837,0.466385,0.960807,0.470494,101.050973,235.552648,"Hidden Size=[12], regularizer=0.02, learning_r..."
475,model_3_41_6,0.976493,-58.328550,0.114408,0.976623,0.228804,396.730285,3.068569,0.178138,0.821052,0.478335,0.958773,0.482549,100.949777,235.451451,"Hidden Size=[12], regularizer=0.02, learning_r..."
476,model_3_39_6,0.977405,-61.038435,0.099312,0.921472,0.219928,414.851298,3.120877,0.183345,0.789149,0.468965,0.960372,0.473097,101.028912,235.530586,"Hidden Size=[12], regularizer=0.02, learning_r..."
477,model_3_37_9,0.973543,-61.952159,-0.033524,0.946862,0.257523,420.961374,3.581152,0.589343,0.792396,0.507467,0.953598,0.511938,100.713292,235.214967,"Hidden Size=[12], regularizer=0.02, learning_r..."
